# Efficient ESG evaluation by NLP

チェックリストを用いた評価はESG評価の代表的な手法です。チェックリストとはESGに関する質問のリストです。「E」の中の「気候変動」のチェックリストでは「気候変動リスクを監視する委員会があるか」「気候変動リスクと機会を特定するプロセスがあるか」などが質問として挙げられます。気候変動以外にも水資源や生物多様性といった他のテーマについてもチェックリストがあり、各テーマのチェックスコアを企業の業種などに応じて重みづけしてESGのスコアを計算します。次の図は、評価機関として有名なRobecoSAMの評価プロセスです(2019年にS&Pグローバルに買収されましたが)。  

![robecosam.PNG!](./images/robecosam.PNG)
出典: [ESG格付け方法論の事例](http://www.env.go.jp/policy/j-hiroba/kigyo/R1/ESGkakudukeGirei.pdf)

本デモでは、ESGの「E」の中の「気候変動」について、TCFDに基づいた開示が行われているのかを自然言語処理を用いて評価します。もちろん人間ほど正確ではありませんし、より精度を上げるにはテキスト抽出や評価の方法について手法を改善する必要があります。ただ、どのようなデータ処理を行えばテキストから評価ができるのか体験いただくには十分だと思います。

デモの手順は以下の通りです。

0. Setup: 環境を構築します
1. Prepare: PDFファイル(統合報告書)からテキストを読み込みます。
2. Preprocess: テキスト解析しやすいよう整形します。
3. Retrieve: 質問に関連する箇所を抽出します。単語ベースとベクトルベースの2つを行います。
4. Evaluate: 抽出結果を評価します。

![demo.PNG](./images/demo.PNG)

### 0. Setup

本Notebookを実行するには、必要なライブラリをインストールした環境が必要です。`environment.yml`を右クリックして"Build Conda Environment"を実行してください。この操作は、File > New > Terminalからターミナルを起動し、次のコマンドを実行したのと同じ効果があります。

* `conda env create -f environment.yml`
* `conda activate esg-nlp`

メニューバーのKearnel > Change Kearnel...で`esg-nlp`が選択できるようになっているはずです。表示されるには少し時間がかかります。Kearnelは特定のNotebookを動かすための専用環境のイメージです。もし選択できない場合、次のコマンドを実行しJupyterがKernelを認識できるようにしてください。

```
ipython kernel install --user --name=esg-nlp
```

`ipython`がインストールされていない場合、次のコマンドでインストールしてください。

* `conda install ipython`

作成したKernelは他のNotebookでも利用できます。本ノートブックをベースに、いろんなテキストを分析してみましょう！

![select_kernel/PNG](./images/select_kernel.PNG)

最後に、テキストを解析するのに必要な言語ごとのモデルをダウンロードします。日本語と英語のモデルをダウンロードしていますが、他の言語のモデルが必要な場合[Install spaCy](https://spacy.io/usage)を参照してください。

In [1]:
!python -m spacy download en_core_web_sm
!python -m spacy download ja_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 52.2 MB/s eta 0:00:0000:0100:01
  Using cached SudachiDict_core-20220519-py3-none-any.whl
  Using cached SudachiPy-0.6.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.2 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')


## 1. Prepare

はじめに、PDFファイルからテキストを読み込みます。デモでは2021年のトヨタの統合報告書を使用します。他のファイルを試してみたい場合、ファイルを`data/raw`のディレクトリに配置した後、下のセルの`file_name`の値を書き替えてください。例えば、2019年のトヨタの統合報告書の場合は`2019_001_annual_en.pdf`になります(2019年の報告書はすでに格納済みです)。

![upload_file.PNG](./images/upload_file.PNG)

In [2]:
# ファイルはdata/raw/のディレクトリに配置してください

file_name = "2021_001_integrated_en.pdf"
# file_name = "2019_001_annual_en.pdf"
# file_name = "2021_001_integrated_jp.pdf"

ファイルの言語を指定します。日本語のファイルを使う場合は"ja"と設定してください。

In [3]:
LANG = "en"
# LANG = "ja"

PDFファイルからのテキスト抽出は、事前に作成しておいた`PDFReader`クラスを使用します。内部的には[`pdfminer.six`](https://github.com/pdfminer/pdfminer.six)を使用しています。実装に興味がある方は[ソースコード](./esg_nlp/data/pdf_reader.py)を参照してください。

In [4]:
import os, sys
sys.path.append("../")
from pathlib import Path
import pandas as pd
import numpy as np
from esg_nlp.data.pdf_reader import PDFReader


path = f"../data/raw/{file_name}"
reader = PDFReader()
df = reader.read_to_frame(path)

PDFファイルを読み込んだ結果を表示します。`page`はページ番号、`order`はページ内でテキストが出現した位置です。

In [5]:
df.head(5)

,page,order,content
0,0,0,Page 1
1,0,1,Integrated Report \n2021\nIntegrated Report 20...
2,1,0,Page 1
3,1,1,Contents
4,1,2,Period Covered\nFiscal 2021 (April 2020 to Mar...


Page1といったページ番号のこ項目や、Contents、といった目次項目まで読み込まれていることがわかります。目次やページ番号はもちろんテキスト解析の対象とはしたくないため、次の`Preprocess`で除去していきます。**PDFの読み込み結果はノイズが非常に多いため、前処理が欠かせません。**

## 2. Preprocess

前処理では、テキストを整形してから不要なテキストを除去するフィルタ処理などを行います。テキストの整形とは、文字コードや文字種の統一などです。テキストの整形をすることで、フィルタ処理が書きやすくなります。具体的には、大文字か小文字かでルールを2つ作るなどの必要がなくなります。

In [6]:
import unicodedata


def preprocess(text):
    if text is None:
        return ""
    _text = text
    _text = unicodedata.normalize("NFKC", _text)  #半角/全角の統一
    _text = _text.replace("\r", "").replace("\n", " ")  # 文中改行の削除
    _text = _text.lower()
    return _text


df["preprocessed"] = df.content.apply(preprocess)

内容が同一の読み取り結果を削除します。例えば、PDFの右上などに表示される目次項目などです。

In [7]:
preprocessed_df = df.drop_duplicates(subset=["preprocessed"])

`Page 1`など、タイトルのみで文を含んでいないセクションを除外します。文の判定は、単純に長さで行います。100は適当な値です。

In [8]:
limit_length = 100

In [9]:
preprocessed_df = preprocessed_df.assign(length=preprocessed_df["preprocessed"].apply(lambda s: len(s)))
preprocessed_df = preprocessed_df[preprocessed_df["length"] > limit_length]

In [10]:
print(f"Rows are decreased from {len(df)} to {len(preprocessed_df)}")

Rows are decreased from 1340 to 249


In [11]:
preprocessed_df.head(10)

,page,order,content,preprocessed,length
4,1,2,Period Covered\nFiscal 2021 (April 2020 to Mar...,period covered fiscal 2021 (april 2020 to marc...,553
7,1,5,icons that link to relevant web pages online.\...,icons that link to relevant web pages online. ...,182
8,1,6,2 \nMessage from the President\n5 \nThe So...,2 message from the president 5 the sourc...,1120
9,1,7,The \nIntegrated Report 2021\n is intended to ...,the integrated report 2021 is intended to co...,317
31,1,29,"On December 14, 2021, Toyota held a briefing o...","on december 14, 2021, toyota held a briefing o...",869
32,1,30,Toyota’s part for carbon neutrality\nAs the op...,toyota’s part for carbon neutrality as the ope...,880
33,1,31,We are living in a diversified world and in an...,we are living in a diversified world and in an...,830
34,1,32,"Five Toyota bZs revealed\nAkio\nToday, we woul...","five toyota bzs revealed akio today, we would ...",709
35,1,33,enabled us to pursue smoothness and maneuverab...,enabled us to pursue smoothness and maneuverab...,855
36,1,34,The important thing is determining to what deg...,the important thing is determining to what deg...,1111


おおむねテキストらしいテキストが残っていることがわかります。

## 3. Retrieve

チェックリストの質問に関係しているセクションを抽出します。単語ベースで行う手法と、ベクトルベースで行う手法を紹介します。

デモではチェックリストの説明として、TCFDの推奨開示を使用します。いかに、日本語と英語の推奨開示事項を示します。

* [気候関連財務情報開示タスクフォース（TCFD）最終報告書](https://www.ecohotline.com/tcfd_compass/download/)
* [The 2017 TCFD recommendations report outlines the TCFD framework for reporting climate-related financial information.](https://www.fsb-tcfd.org/publications/#recommendations)

![tcfd.PNG](./images/tcfd.PNG)
![tcfd_en.PNG](./images/tcfd_en.PNG)

デモとして、戦略のa) を使用します。

In [12]:
if LANG == "ja":
    question = "組織が特定した、短期・中期・長期の気候関連のリスクと機会を記述する。"    
else:
    question = "Describe the climate-related risks and opportunities the organization has identified over the short, medium, and long term."
question = question.lower()

人間が抽出すると、p35の箇所が該当します。

![human_annotation.PNG](./images/human_annotation.PNG)

## 3.1 Word base retrieval

単純に質問文に含まれている単語を含むセクションを抽出します。次の図のように、質問文に含まれる単語がセクションに出現する回数を数えていきます。

![word_based.PNG](./images/word_based.PNG)

文を単語に区切るため[spaCy](https://spacy.io/)を利用します。

In [13]:
import re
from spacy.util import get_lang_class


class Parser():

    def __init__(self, lang):
        self.lang = lang
        self.parser = get_lang_class(self.lang)()
    
    def parse(self, text):
        return self.parser(text)

/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


質問から単語を抽出します。この時、ストップワードと呼ばれる一般的すぎる単語は除外します。

In [14]:
parser = Parser(LANG)
question_words = [str(t).strip() for t in parser.parse(question) if not t.is_stop and not re.match("\'|\.|\?|\/|\,|\-|、|・|。", t.text)]
question_words

['describe',
 'climate',
 'related',
 'risks',
 'opportunities',
 'organization',
 'identified',
 'short',
 'medium',
 'long',
 'term']

セクションごとにキーワードが含まれる数をカウントします。

In [15]:
def count_keyword_match(text, keywords):
    count = 0
    for k in keywords:
        if k in text:
            count += 1
    return count


preprocessed_df = preprocessed_df.assign(
    keyword_match=preprocessed_df["preprocessed"].apply(
        lambda s: count_keyword_match(s, question_words))
)

In [16]:
matched_df = preprocessed_df[preprocessed_df["keyword_match"] > 0]
matched_df.sort_values(by=["keyword_match"], ascending=False).head(5)

,page,order,content,preprocessed,length,keyword_match
722,32,16,4\n communication with stakeholders such as s...,4 communication with stakeholders such as sh...,2233,7
791,35,9,Strategy\nToyota Environmental Challenge 2050\...,strategy toyota environmental challenge 2050 t...,2743,7
790,35,8,Toyota endorsed and signed on to the recom-\nm...,toyota endorsed and signed on to the recom- me...,1819,7
717,32,11,"4\n Besides the Board of Directors meetings, ...","4 besides the board of directors meetings, s...",3299,5
824,36,7,driving mode fuel efficiency) and the developm...,driving mode fuel efficiency) and the developm...,2059,5


p35の文がきちんととれていることがわかります。これにより、キーワードの一致数が多い、つまり重要なセクションにフォーカスして調査をすることが可能になります。

## 3.2 Vector base retrieval

キーワードだけでなく、文章の意味を考慮したセクションの抽出を行います。文章をベクトルとして表現し、ベクトル間の距離で意味の近さを判定します。次の図は、文書中の単語をベクトル化して平均をとることで文書ベクトルを作成しています。文書ベクトルについては「[はじめての自然言語処理 第1回 類似文書検索の手法と精度比較](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part1.html)」などをご参照ください。

![vector_based.PNG](./images/vector_based.PNG)

単語をベクトル化することで、意味の近さを幾何学的に表すことができます(次の図は「[Creating Word Embeddings: Coding the Word2Vec Algorithm in Python using Deep Learning](https://towardsdatascience.com/creating-word-embeddings-coding-the-word2vec-algorithm-in-python-using-deep-learning-b337d0ba17a8)」より引用しました)。

![word_vector_image.png](./images/word_vector_image.png)

単語、また文章をベクトル化する手法はいくつかあります。今回は[BERT](https://blog.google/products/search/search-language-understanding-bert/)と呼ばれる手法を使用します。著名な検索エンジンでも使用されている代表的な手法です。[Hugging Face](https://huggingface.co/)というライブラリで比較的簡単に扱うことができます。

セクションは非常に長いので、文単位に分割します。なお、文を単語に分割した数が512以上になる場合は無視されます(詳細は[Handling multiple sequences](https://huggingface.co/course/chapter2/5?fw=tf#longer-sequences)を参照してください)。

In [17]:
sentences = []
for i, row in preprocessed_df.iterrows():
    c = row["preprocessed"]
    for j, s in enumerate(c.replace("•", ".").replace(";", ".").split(".")):
        sentences.append({
            "page": row["page"],
            "order": row["order"],
            "sentence_order": j,
            "sentence": s[:512],
            "length": len(s)
        })

sentences_df = pd.DataFrame(sentences)
sentences_df.head(5)

,page,order,sentence_order,sentence,length
0,1,2,0,period covered fiscal 2021 (april 2020 to marc...,53
1,1,2,1,some initiatives in fiscal 2022 (april to dec...,75
2,1,2,2,scope of report initiatives and activities of...,110
3,1,2,3,", in japan and overseas reference guidelines t...",183
4,1,2,4,about the pdf this file is an interactive pdf...,105


In [18]:
len(sentences_df)

2429

BERTのモデルを使用し、文をベクトル表現に変換します。文をベクトルに変換するため、事前に作成しておいた`encode`関数を使用します。

In [19]:
from esg_nlp.model.encoder import encode

In [20]:
if LANG == "ja":
    # 参考: https://github.com/BandaiNamcoResearchInc/DistilBERT-base-jp/blob/main/docs/GUIDE.md
    model_name = "bandainamco-mirai/distilbert-base-japanese"
    pretrained_tokenizer_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
else:
    model_name = "distilbert-base-uncased"
    pretrained_tokenizer_name = ""

embeddings = encode(model_name,sentences_df["sentence"].values.tolist(),
                    pretrained_tokenizer_name=pretrained_tokenizer_name)

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


  0%|          | 0/243 [00:00<?, ?it/s]/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 243/243 [02:47<00:00,  1.45it/s]


In [21]:
embeddings.shape

(2429, 768)

それぞれの文に対し、768次元のベクトルが得られました。質問文もベクトルにします。

In [22]:
query = encode(model_name, question, pretrained_tokenizer_name=pretrained_tokenizer_name)
query = np.reshape(query, (1, -1))

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


In [23]:
query.shape

(1, 768)

質問文のベクトルと、各文のベクトルとの距離を算出します。距離はコサイン類似度を用います。1に近いほど似ていて、-1に近いほど似ていません。

In [24]:
from sklearn.metrics.pairwise import cosine_similarity


distance = cosine_similarity(query, embeddings)
measured_df = sentences_df.assign(distance=distance.flatten())
measured_df.sort_values("distance", ascending=False, inplace=True)
measured_df.head(10)

,page,order,sentence_order,sentence,length,distance
1964,43,7,0,through this process of bcp formulation and r...,130,0.876025
1504,35,9,6,"in accordance with this understanding, we hav...",238,0.871666
1500,35,9,2,"among these risks and opportunities, climate ...",176,0.867956
1498,35,9,0,strategy toyota environmental challenge 2050 t...,406,0.867128
957,22,8,11,"to realize carbon neutrality, we need to expa...",171,0.866645
1495,35,8,5,"moreover, the sustainability meeting, which ...",348,0.864610
1558,36,7,8,risk management relating to climate change we...,256,0.855451
1755,39,10,1,working together with suppliers on risk monit...,181,0.850521
723,17,9,2,by presenting a guideline in the product plann...,145,0.848186
1739,38,23,2,"to adapt to recent environmental changes, ats...",71,0.847480


p35以外の、あまり関係ない箇所の文書もリストアップされています。厳密に文を取りたい場合にはノイズが多くなりそうです。

文ごとの近さを、セクション単位に集約します。

In [25]:
distance_df = measured_df.groupby(["page", "order"]).median()["distance"].reset_index()
evaluation_df = pd.merge(preprocessed_df, distance_df, how="inner", on=["page", "order"])
evaluation_df.sort_values("distance", ascending=False).head(5)

,page,order,content,preprocessed,length,keyword_match,distance
159,35,15,Purchasing & production\nLarge-scale introduct...,purchasing & production large-scale introducti...,167,0,0.824555
155,35,8,Toyota endorsed and signed on to the recom-\nm...,toyota endorsed and signed on to the recom- me...,1819,7,0.819167
180,38,23,"affiliates, as an information security framewo...","affiliates, as an information security framewo...",1895,2,0.800432
131,31,8,Fundamental Approach\nToyota regards sustainab...,fundamental approach toyota regards sustainabl...,492,2,0.797904
41,12,6,Corporate Data\nQuickly Adapting to Changes in...,corporate data quickly adapting to changes in ...,108,0,0.796639


## 4. Evaluate

TCFDの推奨事項に近いセクションを、単語ベース、ベクトルベースの2つで抽出してきました。抽出に使用したスコアは、評価にも利用できます。

単語の一致、距離をいずれも0から1へのスコアに変換して評価をしてみます。

* 単語の一致: キーワードリストの個数で割って0~1に
* 距離: マイナスの値は0にして0~1に

In [26]:
evaluation_df["keyword_score"] = evaluation_df["keyword_match"] / len(question_words)
evaluation_df["similarity_score"] = evaluation_df["distance"].clip(lower=0)

In [27]:
evaluation_df.head(5)

,page,order,content,preprocessed,length,keyword_match,distance,keyword_score,similarity_score
0,1,2,Period Covered\nFiscal 2021 (April 2020 to Mar...,period covered fiscal 2021 (april 2020 to marc...,553,0,0.721724,0.000000,0.721724
1,1,5,icons that link to relevant web pages online.\...,icons that link to relevant web pages online. ...,182,0,0.634651,0.000000,0.634651
2,1,6,2 \nMessage from the President\n5 \nThe So...,2 message from the president 5 the sourc...,1120,1,0.710247,0.090909,0.710247
3,1,7,The \nIntegrated Report 2021\n is intended to ...,the integrated report 2021 is intended to co...,317,0,0.691350,0.000000,0.691350
4,1,29,"On December 14, 2021, Toyota held a briefing o...","on december 14, 2021, toyota held a briefing o...",869,0,0.729347,0.000000,0.729347


キーワードへの一致を0.8、類似は0.2として重みをつけてスコアを計算してみます。

In [28]:
evaluation_df["score"] = evaluation_df["keyword_score"] * 0.8 + evaluation_df["similarity_score"] * 0.2
evaluation_df.sort_values("score", ascending=False)

,page,order,content,preprocessed,length,keyword_match,distance,keyword_score,similarity_score,score
155,35,8,Toyota endorsed and signed on to the recom-\nm...,toyota endorsed and signed on to the recom- me...,1819,7,0.819167,0.636364,0.819167,0.672924
156,35,9,Strategy\nToyota Environmental Challenge 2050\...,strategy toyota environmental challenge 2050 t...,2743,7,0.784217,0.636364,0.784217,0.665934
142,32,16,4\n communication with stakeholders such as s...,4 communication with stakeholders such as sh...,2233,7,0.780666,0.636364,0.780666,0.665224
152,34,18,Three Pillars\nThe three pillars of Toyota’s f...,three pillars the three pillars of toyota’s fi...,2269,5,0.796029,0.454545,0.796029,0.522842
162,36,7,driving mode fuel efficiency) and the developm...,driving mode fuel efficiency) and the developm...,2059,5,0.766313,0.454545,0.766313,0.516899
...,...,...,...,...,...,...,...,...,...,...
198,42,11,"work at Toyota, regardless of their school or ...","work at toyota, regardless of their school or ...",250,0,0.414510,0.000000,0.414510,0.082902
60,16,7,At a Q&A session with the media after his pres...,at a q&a session with the media after his pres...,101,0,0.411167,0.000000,0.411167,0.082233
190,40,11,Our Challenge \nThe ratio of females in manage...,our challenge the ratio of females in manager...,1070,0,0.400402,0.000000,0.400402,0.080080
151,34,7,2017/32018/32019/32020/32021/3Dividend per sha...,2017/32018/32019/32020/32021/3dividend per sha...,920,0,0.365372,0.000000,0.365372,0.073074


最大のスコアは以下でした。

In [29]:
np.round(evaluation_df.nlargest(1, "score")["score"].item(), 2)

0.67

例えばスコアが0.6を超えていれば開示しているとみなす、とすると自動的にTCFDの開示有無を判断することができます。
どの程度の値を閾値にするかは、業務的な判断となります。ぜひ様々なレポートで試してみてください。なお、スコアは記載されている内容の良し悪し(気候変動対策の良い/悪い)の評価とは異なる点に注意してください。

## Homework

本Notebookをコピーして、次の処理を行ってみてください。

* トヨタの他の年の統合報告書で同じ処理をし、スコアの値の違いを分析する。
* トヨタ以外の会社の統合報告書で同じ処理を実行し、スコアの値の違いを分析する。

行ってみた結果と得られた知見を、`HomeworkTemplate.pptx`をダウンロードし、記入ください(後日発表頂きます)。
※ダウンロードは右クリックからできます。

![download_file.PNG](./images/download_file.PNG)

### AWSとの接続

SageMaker Studio Labから、API(SDK)を通じてAmazon S3に配置されているファイルにアクセスすることができます。AWS Data Exchangeから購入したデータやS3に格納されている社内のデータを分析することも可能です。

※Studio Lab内のファイルに外部からアクセスすることはできませんが、Studio Labを通じてお客様がファイルをダウンロードすることは可能です。データの利用規約やセキュリティ的に問題がないかはお客様内の情報セキュリティポリシーに沿い判断をお願い致します。

### AWSのサービスの利用

本日ご紹介した実装を本格的に行い業務で使用するのは大変です。AWSのマネージドサービスを利用することで、実装の負荷を下げることができます。

AWSでは、文書検索のサービスを提供しておりこれらを利用することでRetrieveの処理を簡単に実装することができます。

* [Amazon OpenSearch Service](https://aws.amazon.com/jp/opensearch-service/?nc=bc&pg=wik)
* [Amazon Kendra](https://aws.amazon.com/jp/kendra/)

データを保管するいわゆるデータレイクを効率的に構築できるサービスも提供しています。

* [AWS Lake Formation](https://aws.amazon.com/jp/lake-formation/?whats-new-cards.sort-by=item.additionalFields.postDateTime&whats-new-cards.sort-order=desc)
